## Прогнозирование стоимости автомобиля по характеристикам
*Решение на основе шаблона (Baseline) к этому соревнованию*

Основная работа проделана по сбору данных с сайта auto.ru
Предобработка данных заключалась в максимально возможном переводе признаков в числовые
(объем двигателя, мощность) и парсингу комплектации.
Модель машинного обучения: стекинг CatBoost и RandomForest,
в качестве метамодели - LinearRegression.



Обучающий датасет собран с помощью следующего кода:
https://github.com/AnnaGrechina/SkillFactory/blob/master/RDS_3_CarPrice/auto_ru_Parcing.ipynb

In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import sys
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
from tqdm.notebook import tqdm
from catboost import CatBoostRegressor
import re

from sklearn.base import clone
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression


In [2]:
print('Python       :', sys.version.split('\n')[0])
print('Numpy        :', np.__version__)

Python       : 3.7.7 (default, May  6 2020, 11:45:54) [MSC v.1916 64 bit (AMD64)]
Numpy        : 1.18.1


In [3]:
# зафиксируем версию пакетов, чтобы эксперименты были воспроизводимы:
!pip freeze > requirements.txt

In [4]:
# фиксируем RANDOM_SEED, чтобы эксперименты были воспроизводимы!
RANDOM_SEED = 42

# Setup

In [5]:
VERSION    = 5
DIR_TRAIN  = '' # в отличие от Kaggle, данные в основной директории лежат
DIR_TEST   = ''
VAL_SIZE   = 0.1   
N_FOLDS    = 5

# CATBOOST
ITERATIONS = 6000
LR         = 0.05

# Data

In [6]:
train = pd.read_csv(DIR_TRAIN+'BMW_train.csv') # мой подготовленный датасет для обучения модели
test = pd.read_csv(DIR_TEST+'test.csv')
sample_submission = pd.read_csv(DIR_TEST+'sample_submission.csv')

## Data Preprocessing

In [7]:
# удаляем записи с пропусками, заполняем малозначимые недостающие данные
train = train.dropna(subset = ['price', 'name'])
train['Владельцы'].fillna('3 или более', inplace = True)
train['ПТС'].fillna('Оригинал', inplace = True)

In [8]:
def preproc_data(df_input):
    '''includes several functions to pre-process the predictor data.'''
    
    df_output = df_input.copy()
    
    # ################### Предобработка ############################################################## 
    # убираем малозначащие для модели признаки,а также 'vehicleConfiguration' - информация дублируется в других признаках
    df_output.drop(['id', 'Таможня', 'Состояние', 'vehicleConfiguration'], axis=1, inplace=True,)
    
    
    # в явном виде переводим в числовые данные объем двигателя и мощность
    df_output['enginePower'] = df_output['enginePower'].apply(lambda x: int(x[:-4]))
    df_output['engineDisplacement'] = df_output['engineDisplacement'].apply(lambda x: 
                                                                            0 if x == 'undefined LTR' else 10 * float(x[:-4]))

    # ################### fix ############################################################## 
    # Переводим признаки из float в int (иначе catboost выдает ошибку)
    for feature in ['modelDate', 'numberOfDoors', 'mileage', 'productionDate', 'enginePower', 'engineDisplacement']:
        df_output[feature]=df_output[feature].astype('int32')
    

    
    # ################### Feature Engineering ####################################################
    # Добавим признак = количеству опций
    df_output['lenConfiguration'] = df_input['Комплектация'].apply(lambda x: len(configuration_parsing(x)))
    
    # ################### Clean #################################################### 
    # убираем признаки оставшиеся необработанные признаки и исходную комлектацию 
    df_output.drop(['description', 'Владение', 'Комплектация'], axis=1, inplace=True,)
    
    
    return df_output

###################################################

# парсинг конфигурации автомобиля
def configuration_parsing(txt):
    configuration = []
    pattern = r'\"values\":\[(.+?)\]}'
    for txt_elem in re.findall(pattern, txt):
        pattern_txt_elem = r'\"(.+?)\"'
        for config_item in re.findall(pattern_txt_elem, txt_elem):
            configuration.append(config_item)
    return configuration

In [9]:
# код для one-hot-encoding данных комплектации
set_configuration = set()
for i in range(train.shape[0]):
    txt = train['Комплектация'].iloc[i]
    lst = configuration_parsing(txt)
    set_configuration.update(set(lst))

for i in range(test.shape[0]):
    txt = test['Комплектация'].iloc[i]
    lst = configuration_parsing(txt)
    set_configuration.update(set(lst))
    
train = train.reindex(columns = train.columns.tolist() + list(set_configuration))
test = test.reindex(columns = test.columns.tolist() + list(set_configuration))

for col in set_configuration:
    train[col] = train['Комплектация'].apply(lambda x: 1 if col in configuration_parsing(x) else 0)
    test[col] = test['Комплектация'].apply(lambda x: 1 if col in configuration_parsing(x) else 0)

In [10]:
train_preproc = preproc_data(train)
X_sub = preproc_data(test)

train_preproc.drop(['URL'], axis=1, inplace=True,) # лишний столбец, которого нет в testе

X = train_preproc.drop(['price'], axis=1,)
# на основе экспериментов, введем коэффициент, учитывающий изменение экономической ситуации с момента собрания тестового датасета 
# и момента формирования тренировочного датасета

y = 0.95 * train_preproc.price.values 

In [13]:
# отметим категориальные признаки
cat_features_ids = ['bodyType', 'brand', 'color', 'fuelType', 'name',
         'vehicleTransmission', 'Привод', 'Руль', 'Владельцы', 'ПТС']

## Stacking 

In [14]:
def compute_meta_feature(model, X_train, X_test, y_train, cv):
    """
    Computes meta-features usinf the classifier cls
    
    :arg model: scikit-learn classifier
    :arg X_train, y_train: training set
    :arg X_test: testing set
    :arg cv: cross-validation folding
    """
    
    X_meta_train = np.zeros_like(y_train, dtype = np.float32)
    for train_fold_index, predict_fold_index in cv.split(X_train):
        X_fold_train, X_fold_predict = X_train[train_fold_index], X_train[predict_fold_index]
        y_fold_train = y_train[train_fold_index]
        y_fold_test = y_train[predict_fold_index]
        
        folded_model = clone(model)
        folded_model.fit(X_fold_train, y_fold_train)
        X_meta_train[predict_fold_index] = folded_model.predict(X_fold_predict)
        
    meta_model = clone(model)
    meta_model.fit(X_train, y_train)
    
    X_meta_test = meta_model.predict_proba(X_test)[:,1]
    
    return X_meta_train, X_meta_test

In [15]:
n_foldes = 5
cv = KFold(n_splits=n_foldes, shuffle=True)


X_meta_train_features = []
X_meta_test_features = []

# 1 - catboost

model = CatBoostRegressor(iterations = ITERATIONS,
                          learning_rate = LR,
                          random_seed = RANDOM_SEED,
                          eval_metric='MAPE',
                          custom_metric=['R2', 'MAE']
                         )

X_meta_train = np.zeros_like(y, dtype = np.float32)
X_meta_test = np.zeros(len(X_sub), dtype = np.float32)
for train_fold_index, predict_fold_index in cv.split(X):
    X_fold_train, X_fold_predict = X.iloc[train_fold_index], X.iloc[predict_fold_index]
    y_fold_train = y[train_fold_index]
    y_fold_test = y[predict_fold_index]

    folded_model = clone(model)
    folded_model.fit(X_fold_train, y_fold_train,
                     cat_features=cat_features_ids,
                     eval_set=(X_fold_predict, y_fold_test),
                     verbose_eval=1000,
                     use_best_model=True,
                     plot=False
)
    X_meta_train[predict_fold_index] = folded_model.predict(X_fold_predict)
    X_meta_test += folded_model.predict(X_sub)
    

X_meta_test = X_meta_test / n_foldes

X_meta_train_features.append(X_meta_train)
X_meta_test_features.append(X_meta_test)




0:	learn: 1.1877656	test: 1.1983567	best: 1.1983567 (0)	total: 309ms	remaining: 30m 51s
1000:	learn: 0.1326998	test: 0.1433307	best: 0.1433307 (1000)	total: 58.1s	remaining: 4m 50s
2000:	learn: 0.1212541	test: 0.1405281	best: 0.1405120 (1999)	total: 1m 51s	remaining: 3m 43s
3000:	learn: 0.1134092	test: 0.1391019	best: 0.1390077 (2950)	total: 2m 44s	remaining: 2m 44s
4000:	learn: 0.1071498	test: 0.1386272	best: 0.1386193 (3991)	total: 3m 37s	remaining: 1m 48s
5000:	learn: 0.1014630	test: 0.1385299	best: 0.1382949 (4913)	total: 4m 31s	remaining: 54.2s
5999:	learn: 0.0968663	test: 0.1386416	best: 0.1382949 (4913)	total: 5m 25s	remaining: 0us

bestTest = 0.1382949028
bestIteration = 4913

Shrink model to first 4914 iterations.
0:	learn: 1.2024224	test: 1.1086331	best: 1.1086331 (0)	total: 66.2ms	remaining: 6m 37s
1000:	learn: 0.1315436	test: 0.1339522	best: 0.1339343 (999)	total: 57.6s	remaining: 4m 47s
2000:	learn: 0.1201408	test: 0.1307653	best: 0.1307201 (1962)	total: 1m 56s	remaining: 

In [16]:
# 2 - randomForestRegressor

model = RandomForestRegressor(n_estimators=300, random_state=42)

X_meta_train = np.zeros_like(y, dtype = np.float32)
X_train_num = X.drop(cat_features_ids, axis = 1)
X_sub_num = X_sub.drop(cat_features_ids, axis = 1)

for train_fold_index, predict_fold_index in cv.split(X_train_num):
    X_fold_train, X_fold_predict = X_train_num.iloc[train_fold_index], X_train_num.iloc[predict_fold_index]
    y_fold_train = y[train_fold_index]

    folded_model = clone(model)
    folded_model.fit(X_fold_train, y_fold_train)
    X_meta_train[predict_fold_index] = folded_model.predict(X_fold_predict)

meta_model = clone(model)
meta_model.fit(X_train_num, y)

X_meta_test = meta_model.predict(X_sub_num)

X_meta_train_features.append(X_meta_train)
X_meta_test_features.append(X_meta_test)


In [17]:
stacked_features_train = np.vstack(X_meta_train_features).T
stacked_features_test = np.vstack(X_meta_test_features).T

In [18]:
stacked_features_test[:10,:]

array([[1621740.        , 1620281.99366667],
       [2485411.        , 2834278.53972222],
       [1323980.25      , 1346865.66666667],
       [2427400.75      , 2437585.99683333],
       [5008473.        , 4769796.76183333],
       [2023370.375     , 2116220.703     ],
       [1090368.875     , 1006932.93633333],
       [ 680486.1875    ,  707463.41666667],
       [1444184.875     , 1481695.47222222],
       [1398908.25      , 1331333.16666667]])

In [20]:
final_model = LinearRegression()
final_model.fit(stacked_features_train, y)
sample_submission['price'] = np.floor(final_model.predict(stacked_features_test) / 10000) * 10000 
sample_submission.to_csv(f'submission_stack_v{VERSION}_BMW.csv', index=False)
sample_submission.head(10)


,id,price
0,0,1620000.0
1,1,2530000.0
2,2,1320000.0
3,3,2430000.0
4,4,5000000.0
5,5,2030000.0
6,6,1080000.0
7,7,680000.0
8,8,1450000.0
9,9,1390000.0
